# Amazon SageMaker Real-Time Hosting with Whisper Transcription

This notebook show's how to use [SageMaker's real-time inference endpoints](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html) to host [OpenAI's Whisper](https://github.com/openai/whisper) model for audio-to-text transcription in real time. In this notebook you will...

1. Install the whisper library
2. Load a whisper model
3. Run inference locally on an example audio dataset
4. Serialize the whisper model to S3
5. Create a SageMaker model
6. Deploy the SageMaker model to a real-time endpoint
7. Run inference on the SageMaker endpoint
8. Tear down the SageMaker endpoint

# Install Whisper and Import Libraries

The cells below will install the Python packages needed to use Whisper models and evaluate the transcription results.

In [ ]:
%pip install -U openai-whisper
%pip install torchaudio --quiet

In [1]:
import os
import multiprocessing
import numpy as np
import torch
import pandas as pd
import whisper
import sagemaker
import time
from tqdm.notebook import tqdm
import boto3

DEVICE = "cuda" #if torch.cuda.is_available() else "cpu"

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/edenduthie/Library/Application Support/sagemaker/config.yaml


# Run example inference locally using a base Whisper model

Now that the dataset has been created, we can download a whisper model using the `whisper.load_model` function. In this example we will be using the `whisper-large-v3` model, but there are larger models also available for download. Once the model is downloaded, you will use it to run an illustrative example inference call locally on the notebook before you deploy this model to a SageMaker endpoint.

In [ ]:
model = whisper.load_model("large-v3")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

In [2]:
from scipy.signal import resample
import numpy as np
import base64

def convert_resample_8k(int16_array):
    normalized_arr = int16_array / 32768.0
    float_arr = normalized_arr.astype(np.float32)
    return resample(float_arr,float_arr.size*2)

def byte64_to_numpy(payload):
    payload = base64.b64decode(payload)
    return np.frombuffer(payload, dtype=np.int16)


#with open('audio_track_1.raw', 'rb') as file:
#    data = file.read()

#encoded_data = base64.b64encode(data)


#audio = convert_resample_8k(byte64_to_numpy(encoded_data))
#audio = torch.from_numpy(audio)


In [3]:
audio = whisper.pad_or_trim(audio.flatten()).to(DEVICE)
mel = whisper.log_mel_spectrogram(audio,n_mels=128)
options = whisper.DecodingOptions(language="en", without_timestamps=True, fp16 = False)
out = model.decode(mel, options)
print(f'Example Transcription: \n{out.text}')

NameError: name 'audio' is not defined

# SageMaker Inference

In this section, you will deploy the whisper model from the previous section to a real time API endpoint on Amazon SageMaker. You start this section by instantiating a sagemaker session and defining a path in Amazon S3 for your model artifacts shown below.

In [5]:
alliance = boto3.session.Session(profile_name='alliance')
sess = sagemaker.session.Session(boto_session=alliance)
bucket = sess.default_bucket()
prefix = 'lissten-demo-deploy/'
s3_uri = f's3://{bucket}/{prefix}'

## Create Model Artifacts in S3

You can now take the whisper model which was loaded previously and save it using PyTorch. Make sure you save both a model state as well as model dimensions to be compatible with the whisper library.

In [ ]:
torch.save(
    {
        'model_state_dict': model.state_dict(),
        'dims': model.dims.__dict__,
    },
    'whisper-large-v3.pt'
)

Once the model has been saved, you will package the model into a tar.gz file and upload it to Amazon S3. This serialized model will be the model artifact which is referenced for real-time inference.

In [6]:
!mkdir -p model
!touch model/whisper-large-v3.pt
!cd model && tar -czvf model.tar.gz whisper-large-v3.pt
!mv model/model.tar.gz .
!tar -tvf model.tar.gz
model_uri = sess.upload_data('model.tar.gz', bucket = bucket, key_prefix=f"{prefix}model")
!rm model.tar.gz
!rm -rf model

a whisper-large-v3.pt
-rwx------  0 edenduthie staff       0 Jul  1 16:39 whisper-large-v3.pt


In [7]:
model_uri

's3://sagemaker-ap-southeast-2-471112788437/lissten-demo-deploy/model/model.tar.gz'

## Create SageMaker Model Object

Once the model artifact has been uploaded to S3, you will use the SageMaker SDK to create a `model` object which references the model artifact in S3, one of SageMaker's PyTorch inference containers, and the inference code stored in the `src` directory in this repository. The `inference.py` is the code which is executed at runtime while the `requirements.txt` tells SageMaker to install the `whisper` library inside its Docker container.

In [8]:
#role = sagemaker.get_execution_role()
role = "arn:aws:iam::471112788437:role/SageMakerExecutionRole" # TODO don't hard-code this

image = sagemaker.image_uris.retrieve(
    framework='pytorch',
    region='ap-southeast-1',
    image_scope='inference',
    version='2.1',
    instance_type='ml.g5.2xlarge',
    sagemaker_session=sess,
)

model_name = f'lissten-model'
whisper_model_sm = sagemaker.model.Model(
    model_data=model_uri,
    image_uri=image,
    role=role,
    entry_point="inference.py",
    source_dir='src',
    name=model_name,
    sagemaker_session = sess,
)

In [9]:
print(image)

763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/pytorch-inference:2.1-gpu-py310


## Deploy to a Real Time Endpoint

Deploying the `model` object to sagemaker can be done with the `deploy` function. Notice that you will be using a `ml.g4dn.xlarge` instance type in order to take advantage of a AWS's low cost GPU instances for accelerated inference.

In [10]:
endpoint_name = f'lissten-endpoint'
whisper_model_sm.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
    endpoint_name=endpoint_name,
    wait=False,
    sagemaker_session=sess,
)

ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Region of "763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/pytorch-inference:2.1-gpu-py310" does not match expected region of ap-southeast-2. Cross region ECR image pulls are not allowed.

In [45]:
endpoint_name

'whisper-endpoint'

In [29]:
whisper_endpoint = sagemaker.predictor.Predictor(endpoint_name)
assert whisper_endpoint.endpoint_context().properties['Status'] == 'InService'

In [30]:

with open('audio_track_1.raw', 'rb') as file:
    data = file.read()

encoded_data = base64.b64encode(data)



In [33]:
out = whisper_endpoint.predict(encoded_data)
print(f'Example Transcription: \n{out}')

Example Transcription: 
b'Please tell me in a few words how I can help you today.'


## Sequential Latency Test

You can also run a latency test to see how fast the g4dn instance is able to process single input requests. The first cell will ensure the instance is warmed and the next cell will time the requests coming into the endpoint.

In [ ]:
# warm up the instance
for i in range(10):
    out = whisper_endpoint.predict(encoded_data)

In [ ]:
%%timeit
out = whisper_endpoint.predict(encoded_data)

## Optional: Clean Up Endpoint

Once you have finished testing you endpoint, you have the option to delete your SageMaker endpoint. This is a good practice as experimental endpoints can be removed in order to decrease your SageMaker costs when they are not in use.

In [ ]:
#whisper_endpoint.delete_endpoint()